In [ ]:
# Get Data
!pip install vaderSentiment
!pip install wordcloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
import matplotlib.dates as mdates
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem.porter import *
from wordcloud import WordCloud

In [ ]:
data = pd.read_csv("Dataset.csv", encoding='ISO-8859-2')
data.head()

In [ ]:
# Remove pattern function
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, '', input_txt)
  return input_txt

In [ ]:
# Remove emoticons 
data['Message'] = data['Message'].str.replace("[^\w()'+]", " ")
data

In [ ]:
# Remove links
data['Message'] = data['Message'].str.replace("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*,]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ")
data.head()

In [ ]:
# Remove special characters, numbers, punctuations (except exclamation mark)
data['Message'] = data['Message'].str.replace("[^a-zA-Z#!]", " ")
data.head()

In [ ]:
# Remove short words
data['Message'] = data['Message'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
data.head()


In [ ]:

# Tokenize Data
tokenize_meesage = data['Message'].apply(lambda x: x.split())
tokenize_meesage.head()

In [ ]:
# Stemming Data

stemmer = PorterStemmer()

tokenize_meesage = tokenize_meesage.apply(lambda x: [stemmer.stem(i) for i in x])
tokenize_meesage.head()

In [ ]:
for i in range(len(tokenize_meesage)):
  tokenize_meesage[i] = ' '.join(tokenize_meesage[i])

data['Message'] = tokenize_meesage
data.head()

In [ ]:

# WordCloud
all_words = ' '.join([text for text in data['Message']])

wordcloud = WordCloud(width=800, height=500, random_state=21, background_color='white',  max_font_size=110).generate(all_words)

plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


In [ ]:
# Hashtags mentions

def hashtag_extract(x):
  hashtags = []

  for i in x:
    ht = re.findall(r"#(\w+)", i)
    hashtags.append(ht)

  return hashtags

# Extract hashtags

hashtags = hashtag_extract(data['Message'])

# Unnesting list

hashtags = sum(hashtags, [])

In [ ]:
# Calculate Polarity

analyser = SentimentIntensityAnalyzer()

scores = []
for sentence in data['Message']:
  score = analyser.polarity_scores(sentence)
  scores.append(score)

scores = pd.DataFrame(scores)

data['Compound'] = scores['compound']
data['Negative'] = scores['neg']
data['Positive'] = scores['pos']
data.head()

In [ ]:
# Calculate Mentions

def word_extract(x):
  words = []

  for i in x:
    ht = re.findall(r"(\w+)", i)
    words.append(ht)

  return words

# Extract words

words = word_extract(data['Message'])

# words

# Unnesting list

words = sum(words, [])

In [ ]:
# Frequency Distribution

a = nltk.FreqDist(words)

b = pd.DataFrame({'Word': list(a.keys()), 'Count': list(a.values())})
length = len(b)

# Top 10 Most Frequent Words
b = b.nlargest(columns="Count", n = 10)
plt.figure(figsize=(10,5))
axis = sns.barplot(data=b, x='Word', y='Count')
axis.set(ylabel='Count')
plt.title('Top 10 Most Frequent Words')
plt.show()

In [ ]:
# Set type of polarity
polarity = []

for i in range(len(data)):
  if (data['Compound'][i] < 0):
    polarity.append("Negative")
  elif (data['Compound'][i] > 0):
    polarity.append("Positive")

polarity = pd.DataFrame(polarity)
data['Sentiment'] = polarity
data.head()

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
# import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
# classify the comments
summary = {"positive":0,"negative":0}

for x in words:
    ss = sid.polarity_scores(x)
    if ss["compound"] >= 0.0:
        summary["positive"] +=1
        postot=summary["positive"]
    else:
        summary["negative"] +=1
        negtot=summary["negative"]

summary

In [ ]:
import matplotlib.pyplot as plt
labels = 'Positive', 'Negative'
sizes = [postot, negtot]

explode = (0.1, 0)  

fig, ax = plt.subplots()
ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.show()

In [ ]:
if postot>negtot:
  print("Congratulations on Your Positive Sentiment Analysis Result!")
  print("Hi Dear,")
  print("I hope this message finds you well! this is the result of your recent sentiment analysis result is overwhelmingly positive! Your positivity shines through, and it's truly uplifting.")
  print("This positive sentiment reflects not only a great attitude but also contributes to creating a positive and supportive environment. Keep up the excellent work, and continue spreading those good vibes.")
  print("Your positivity is contagious and makes a positive impact on those around you. Once again, congratulations on the positive sentiment analysis. Wishing you continued success and positivity in all that you do!")
else:
  print("Hi Dear,")
  print("I hope this message finds you well, though I understand that your recent sentiment analysis result suggests a negative sentiment. I want you to know that we here for you, and your feelings are valid.")
  print("Life can present challenges, and it's okay to seek support. Considering the negative sentiment, I would like to gently suggest the idea of speaking with a therapist. Professional support can offer a safe and confidential space to explore your thoughts and feelings.")
  print("Therapists are trained to provide guidance and support for various life challenges, and reaching out to one doesn't mean you're alone—it means you're taking a positive step toward your well-being. ")
  print("Remember, taking care of your mental health is an essential part of self-care. You don't have to face difficulties alone, and seeking help is a sign of strength.")
  print("Take care,")